In [1]:
import json
from collections import Counter

import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

### [Question 1](#Question 1)

What are the top 3 and the bottom 3 states in terms of number of users?

In [6]:
with open('song.json') as jsonfile:
    data = json.load(jsonfile)


In [27]:
df = pd.DataFrame(data)
df.head()
len(df['user_id'].unique())
len(df)

4000

In [50]:
len(df.loc[df['user_state'] == 'New York']['user_id'].unique()
)

23

In [46]:
df.head()

,id,song_played,time_played,user_id,user_sign_up_date,user_state
0,GOQMMKSQQH,Hey Jude,2015-06-11 21:51:35,122,2015-05-16,Louisiana
1,HWKKBQKNWI,We Can Work It Out,2015-06-06 16:49:19,3,2015-05-01,Ohio
2,DKQSXVNJDH,Back In the U.S.S.R.,2015-06-14 02:11:29,35,2015-05-04,New Jersey
3,HLHRIDQTUW,P.s. I Love You,2015-06-08 12:26:10,126,2015-05-16,Illinois
4,SUKJCSBCYW,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,6,2015-05-01,New Jersey


In [79]:
df_state = df.groupby(['user_state','user_id']).size().reset_index(name='user_counts')

df_state.groupby('user_state').count().sort_values(by ='user_id', ascending = False).head()

,user_id,user_counts
user_state,,
New York,23,23
California,21,21
Texas,15,15
Pennsylvania,9,9
Ohio,9,9


<a id='Question 1'></a>
### Answer Q1
The top three states by count are-  New York, California, Texas
The bottom three states by count are - Nebraska, Arizona, Kansas

### [Question 2](#Question 2)

What are the top 3 and the bottom 3 states in terms of user engagement? You can choose how to mathematically deﬁne user engagement. What the CEO cares about here is in which states users are using the product a lot/very little

I think the user_engagment can be defined as how many times the user has listened to music

In [123]:
df['time_played_date'] = pd.to_datetime(df['time_played']).apply(lambda x: x.date())
df['user_sign_up_date'] = pd.to_datetime(df['user_sign_up_date']).apply(lambda x: x.date())
df['active'] = df['time_played_date'] - df['user_sign_up_date']
df['active'] = df['active'] / np.timedelta64(1, 'h')/24

In [137]:
df_date=df.groupby(['user_state','user_id']).size().reset_index(name='date_counts')

In [157]:
df_rank = df_date.groupby('user_state').mean().drop(['user_id'], axis = 1)
# We take the mean of the number of times as song has been played per state per user 

In [161]:
#The more active mean, the more time user takes to play a song since he registered.
df_rank['active_mean'] = df.groupby(['user_state'])['active'].mean()
df_rank['active_mean'] = df_rank['active_mean'].apply(lambda x:1/x)

In [164]:
df_rank.sort_values(by = ['active_mean'], ascending= False)

,date_counts,active_mean
user_state,,
Virginia,8.500000,0.043367
Kansas,8.000000,0.041885
Idaho,26.000000,0.038690
Colorado,18.000000,0.037842
Iowa,23.000000,0.037705
Washington,20.500000,0.037580
Nebraska,36.000000,0.037113
Louisiana,21.000000,0.036219
Oklahoma,24.500000,0.036136


In [ ]:
df_rank.sort_